<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/main/ipynb/Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
except ModuleNotFoundError:
    IN_COLAB = False

Mounted at /content/drive


Check GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Sep 14 10:51:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check TPU

Check Memory

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
!git clone https://github.com/EddyDavies/Trade_With_Twitter.git

Cloning into 'Trade_With_Twitter'...
remote: Enumerating objects: 525, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 525 (delta 301), reused 317 (delta 142), pack-reused 0
Receiving objects: 100% (525/525), 153.47 KiB | 4.80 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [5]:
!ls ../

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [6]:
%cd Trade_With_Twitter/src
!ls

/content/Trade_With_Twitter/src
bert_farm.ipynb  decorators.py	sentiment  utils.py
bert_farm.py	 prices		twitter


In [7]:
!pip install -r ../requirements.txt

     |████████████████████████████████| 2.8 MB 5.1 MB/s 
     |████████████████████████████████| 184 kB 65.7 MB/s 
     |████████████████████████████████| 895 kB 54.0 MB/s 
     |████████████████████████████████| 636 kB 38.0 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 53.9 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=e768bb8558551fb39eb16ac376e19504787bae322b777230ab03f7334786083f
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import os
import pandas as pd
from tqdm import tqdm

from utils import get_date_array, get_month_array
from sentiment.inference import get_paths, get_tweets, get_sentiments, save_sentiments
from utils import get_date_array, get_month_array, check_last_day

In [9]:
# 10% before "2017-09-19" and 30% after
# dates_range = ("2017-01-01", "2021-06-20")
# dates_range = ("2017-09-19", "2021-06-20")
# dates_range = ("2018-01-12", "2021-06-20")
# 100% after "2018-03-19"
# dates_range = ("2018-03-19", "2021-06-20")
dates_range = ("2018-06-29", "2021-06-20")
months_range = ["Jan 17", "Jun 21"]

months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [13]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# model = model.to('cuda:0')

# sentiment_analysis = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)
sentiment_analysis = pipeline("sentiment-analysis", model=model_name, device=0)

In [14]:
if IN_COLAB:
    source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData', model_name=model_name)
else:
    source, results_folder = get_paths()

In [ ]:
for date in dates:
    ids, tweets = get_tweets(date, source)

    percentage_per_chunk = 100
    save_every = 20000
    results = get_sentiments(date, tweets, ids, results_folder,
                              sentiment_analysis=sentiment_analysis,
                              save_every=save_every,
                              percentage_per_chunk=percentage_per_chunk,
                             slice_size=None,
                             batch_size=100)

    # check_last_day(results_folder, date)

2018-08-21:  51%|█████     | 104/205 [00:19<00:20,  5.00it/s]

In [ ]:
!ls ../

In [ ]:
# !rm -r ../data/bitcoin_scores

In [ ]:
print(results_folder)

In [ ]:
!git pull